In [ ]:
import random
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
# Here we import the function for auto-selection of the most active electrodes
codeDirectory = ""
os.chdir(codeDirectory)
from interface.selectElectrodes import getElectrodeListsWithSelection

In [ ]:
from data.utils.cmos_utils import getCoordinates
from matplotlib import style
plt.style.use('classic')
# Importing MaxLab libraries
import maxlab
import maxlab.system
import maxlab.chip
import maxlab.util
import maxlab.saving
import supplementary.stimulusLoopFunctions as stimulusLoopFunctions

In [ ]:
voltagemapPath = ...
configFilePath = None
saveDirectory = ...
filename = "Spontaneous"
duration = 300

In [ ]:
selectionFilename = "selection.npy"
voltagemap = np.load(voltagemapPath) # make sure voltage map is in input folder

recordingElectrodes, electrodesSource = getElectrodeListsWithSelection(voltagemap, saveDirectory, loadFileBool=False, n_sample=1020, selection_threshold=15)

In [ ]:
# Initialize MaxLab system into a defined state
maxlab.util.initialize()
# Correct the zero-mean offset in each electrode reading
maxlab.util.offset()

# Loop through the binary voltage map to attain electrode numbers from the area of interest
electrodes = recordingElectrodes

print("Total number of electrodes in isolated area: " + str(len(electrodes)))

# check that this is less than available amplifiers (1024?)
assert len(electrodes) < 1024
#############################################
# Initialising the chip configuration
#############################################
array = maxlab.chip.Array('array') # 'stimulation' is the token ID to identify the instance on the server
array.reset()
array.clear_selected_electrodes()
# Selecting the electrodes to be routed
if configFilePath is not None and os.path.exists(configFilePath):
    array.load_config(configFilePath)
    array.download()
else:
    array.select_electrodes(electrodes)
    array.route()
    # Download the prepared array configuration to the chip
    array.download()
    if configFilePath is not None:
        array.save_config(configFilePath)
# Code modification for different order settings of voltage array
wells = range(1)
# Investigating the total number of electrodes routed

In [ ]:
Saver = maxlab.saving.Saving()
Saver.open_directory(saveDirectory)
Saver.set_legacy_format(True)
Saver.group_delete_all()

# Comment out for loop in case of spikes only
for well in wells:
    Saver.group_define(well, "routed")

Saver.start_file(filename)
Saver.start_recording(wells)

# Run for however many seconds possible in sequence --> send stimulation pulse for x min
time.sleep(duration)

# Stop recording data
Saver.stop_recording()
Saver.stop_file()
Saver.group_delete_all()